In [3]:
import cv2
import os
from datetime import datetime
import time
import socket
import subprocess
import Helper
import pandas as pd
from super_gradients.training import models
import numpy as np


In [4]:
import socket
def get_ip_address():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) #creat connection
        s.connect(("8.8.8.8", 80)) #ping google
        ip = s.getsockname()[0] #own ip add.
        s.close()
        return ip
    except Exception as e:
        return F"Could not get IP: {e}"

get_ip_address()

'172.20.10.4'

In [5]:
model_path = r"D:\Final Code\Final Project\Yolo nas\Dataset\checkpoints\tips and liquid\RUN_20240120_202748_124739\ckpt_best.pth"

best_model = models.get('yolo_nas_l',
                        num_classes=2,
                        checkpoint_path=model_path)

[2024-02-01 12:45:51] INFO - checkpoint_utils.py - Successfully loaded model weights from D:\Final Code\Final Project\Yolo nas\Dataset\checkpoints\tips and liquid\RUN_20240120_202748_124739\ckpt_best.pth EMA checkpoint.


In [7]:
def start_server():
    keys = ['Tip1', 'Tip2', 'Tip3', 'Tip4', 'Tip5', 'Tip6', 'Tip7', 'Tip8']
    Tips_check = {key: [] for key in keys}
    Liquid_check = {key: [] for key in keys}
    Tips_volume = {key: [] for key in keys}

    keys2 = ['Tips_Count', 'Liquid_Count']
    counts_data = {key: [] for key in keys2}

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    port = 9999
    server_socket.bind(('', port))
    server_socket.listen(1)
    print(f"Server started on port {port}")

    try:
        while True:  # Loop to accept new connections
            client_socket, addr = server_socket.accept()
            print(f"Got a connection from {addr}")
            while True:  # Loop to handle commands from a connected client
                command = client_socket.recv(1024).decode('utf-8').strip().lower()
                if not command:  # Break the loop if the command is empty (client disconnected)
                    break
                if command == 'capture_image':
                    response = Helper.capture_hd_image_with_lock('default_project')
                    client_socket.send(response.encode('utf-8'))

                    predictions = best_model.predict([response], conf=0.7)
                    Helper.save_predictions(predictions, output_folder="output_predictions")

                    class_counts, bounding_boxes, bounding_box_centers, liquid_height_percentages = Helper.process_predictions(predictions)
                    tip_presence, missing_tip_positions = Helper.find_missing_tips(bounding_box_centers)

                    liquid_values = liquid_height_percentages['Liquid']
                    tip_index = 0

                    # Iterate through the 'Liquid' values and assign to 'Tips_volume' based on 'tip_presence'
                    for i, presence in enumerate(tip_presence):
                        tip_key = f'Tip{i + 1}'
                        if presence == 1:
                            Tips_volume[tip_key].append(liquid_values[tip_index])
                            tip_index += 1
                        else:
                            Tips_volume[tip_key].append(0)

                    counts_data['Tips_Count'].append(class_counts['Tip'])
                    counts_data['Liquid_Count'].append(class_counts['Liquid'])

                    for i, presence in enumerate(tip_presence):
                        tip_key = f'Tip{i + 1}'
                        if presence == 1:
                            Tips_check[tip_key].append("Attached")
                        else:
                            Tips_check[tip_key].append("MISSING")

                elif command == 'shutdown_server':
                    print("Shutting down server.")
                    server_socket.close()
                    return Tips_check, Tips_volume, counts_data
                else:
                    print(f"Unknown command: {command}")
    except Exception as e:
        print(f"Server error: {e}")
    finally:
        server_socket.close()
        print("Server has been shut down.")

if __name__ == "__main__":
    Tips_check, Tips_volume, counts_data = start_server()

Server started on port 9999
Got a connection from ('172.20.10.6', 46938)
HD Image captured with locked focus: default_project_20240201124953_hd.jpg


c:\Users\thisi\anaconda3\envs\OT2\lib\site-packages\super_gradients\training\utils\media\image.py:64: UserWarning: It seems you are using predict() with batch input
  warnings.warn("It seems you are using predict() with batch input")
[2024-02-01 12:49:55] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Got a connection from ('172.20.10.6', 46944)
HD Image captured with locked focus: default_project_20240201125004_hd.jpg


[2024-02-01 12:50:06] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Got a connection from ('172.20.10.6', 46950)
HD Image captured with locked focus: default_project_20240201125014_hd.jpg


...

Got a connection from ('172.20.10.6', 47046)
HD Image captured with locked focus: default_project_20240201125351_hd.jpg


[2024-02-01 12:53:53] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Got a connection from ('172.20.10.6', 47052)
HD Image captured with locked focus: default_project_20240201125402_hd.jpg


[2024-02-01 12:54:04] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Got a connection from ('172.20.10.6', 47058)
Shutting down server.
Server has been shut down.


In [8]:
Tips_volume 
Tips_volume1 = pd.DataFrame(Tips_volume)
Tips_volume1.index = [f'ROW {i + 1}' for i in range(len(Tips_volume1))]
Tips_volume1=Tips_volume1.round()
Tips_volume1

,Tip1,Tip2,Tip3,Tip4,Tip5,Tip6,Tip7,Tip8
ROW 1,42.0,0.0,0.0,41.0,42.0,41.0,42.0,42.0
ROW 2,41.0,0.0,0.0,41.0,41.0,41.0,41.0,41.0
ROW 3,41.0,41.0,41.0,42.0,41.0,41.0,39.0,40.0
ROW 4,43.0,42.0,42.0,41.0,42.0,41.0,41.0,41.0
ROW 5,42.0,41.0,41.0,42.0,41.0,42.0,41.0,41.0
ROW 6,41.0,41.0,42.0,41.0,41.0,42.0,41.0,40.0
ROW 7,41.0,41.0,41.0,40.0,41.0,43.0,42.0,40.0
ROW 8,41.0,42.0,41.0,41.0,41.0,43.0,41.0,39.0
ROW 9,41.0,42.0,42.0,41.0,43.0,41.0,41.0,40.0
ROW 10,41.0,41.0,41.0,40.0,42.0,44.0,41.0,40.0


In [9]:
Tips_check = pd.DataFrame(Tips_check)
Tips_check.index = [f'ROW {i + 1}' for i in range(len(Tips_check))]
Tips_check

,Tip1,Tip2,Tip3,Tip4,Tip5,Tip6,Tip7,Tip8
ROW 1,Attached,MISSING,MISSING,Attached,Attached,Attached,Attached,Attached
ROW 2,Attached,MISSING,MISSING,Attached,Attached,Attached,Attached,Attached
ROW 3,Attached,Attached,Attached,Attached,Attached,Attached,Attached,Attached
ROW 4,Attached,Attached,Attached,Attached,Attached,Attached,Attached,Attached
ROW 5,Attached,Attached,Attached,Attached,Attached,Attached,Attached,Attached
ROW 6,Attached,Attached,Attached,Attached,Attached,Attached,Attached,Attached
ROW 7,Attached,Attached,Attached,Attached,Attached,Attached,Attached,Attached
ROW 8,Attached,Attached,Attached,Attached,Attached,Attached,Attached,Attached
ROW 9,Attached,Attached,Attached,Attached,Attached,Attached,Attached,Attached
ROW 10,Attached,Attached,Attached,Attached,Attached,Attached,Attached,Attached


In [10]:
counts_data = pd.DataFrame(counts_data)
counts_data.index = [f'Round {i + 1}' for i in range(len(counts_data))]
counts_data

,Tips_Count,Liquid_Count
Round 1,9,8
Round 2,9,8
Round 3,8,8
Round 4,8,8
Round 5,8,8
Round 6,8,8
Round 7,8,8
Round 8,8,8
Round 9,8,8
Round 10,8,8


In [11]:
writer = pd.ExcelWriter('dataframes_combined.xlsx', engine='xlsxwriter')
Tips_volume1.to_excel(writer, sheet_name='Tips_volume1')
Tips_check.to_excel(writer, sheet_name='Tips_check')
counts_data.to_excel(writer, sheet_name='counts_data')
writer.close()


In [ ]:
#!pip install XlsxWriter